<a href="https://colab.research.google.com/github/helenatong/oc_p4_Anticipez_les_besoins_en_consommation_de_batiments/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GradientBoostingRegressor question sur le gradient

# **Introduction**

---

## **Prérequis**
- Télécharger [dataset nettoyé](https://drive.google.com/file/d/1-EuzI1ibLAG_1brg_v3MKcDtEgKDFIdl/view?usp=drive_link) dans Google Drive
- Modifier le chemin du fichier dans le code


---

---

## **Sommaire**

### **I. Installation et mise en place de l'environnement de travail**

### **II. Définitions des fonctions auxiliaires**

### **III. Prédiction de la variable TotalGHGEmissions**
**A. Entrainement du premier modèle: régression linéaire**

# **I. Installation et mise en place de l'environnement de travail**

In [1]:
# importation des librairies
import numpy as np
import pandas as pd

import zipfile
import os

import re

import sys

import missingno as msno
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression

import datetime

from google.colab import drive

In [2]:
# informations sur l'environnement virtuel
print('Version des librairies :')
print('Python        : ' + sys.version)
print('NumPy         : ' + np.version.full_version)
print('Pandas        : ' + pd.__version__)
print('Matplotlib    : ' + mpl.__version__)
print('Seaborn       : ' + sns.__version__)

now  = datetime.datetime.now().isoformat()
print('Run date      : ' + now)

Version des librairies :
Python        : 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
NumPy         : 1.25.2
Pandas        : 2.0.3
Matplotlib    : 3.7.1
Seaborn       : 0.13.1
Run date      : 2024-05-14T16:38:01.484791


In [3]:
# configurations de l'affichages des données
pd.set_option('display.max_columns', None)

In [4]:
# connexion au drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# vérification de l'importation
df = pd.read_parquet('/content/drive/MyDrive/openclassrooms/project_list/project_4/data/cleaned_2016_Building_Energy_Benchmarking.pq')
df.head(2)

,OSEBuildingID,BuildingType,PrimaryPropertyType,Neighborhood,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,ENERGYSTARScore,SiteEnergyUse(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),TotalGHGEmissions,BuildingAge,LastYearENERGYSTARCertified,NbYearENERGYSTARCertified
0,1,NonResidential,Hotel,DOWNTOWN,1.0,12,88434,0,88434,Hotel,88434.0,missing,0.0,missing,0.0,60.0,7226362.5,2003882.0,3946027.0,1276453.0,249.98,89,0.0,0
1,2,NonResidential,Hotel,DOWNTOWN,1.0,11,103566,15064,88502,Hotel,83880.0,Parking,15064.0,Restaurant,4622.0,61.0,8387933.0,0.0,3242851.0,5145082.0,295.86,20,0.0,0


In [6]:
# vérification des données
df.isna().sum(axis=0)

OSEBuildingID                      0
BuildingType                       0
PrimaryPropertyType                0
Neighborhood                       0
NumberofBuildings                  0
NumberofFloors                     0
PropertyGFATotal                   0
PropertyGFAParking                 0
PropertyGFABuilding(s)             0
LargestPropertyUseType             0
LargestPropertyUseTypeGFA          0
SecondLargestPropertyUseType       0
SecondLargestPropertyUseTypeGFA    0
ThirdLargestPropertyUseType        0
ThirdLargestPropertyUseTypeGFA     0
ENERGYSTARScore                    0
SiteEnergyUse(kBtu)                0
SteamUse(kBtu)                     0
Electricity(kBtu)                  0
NaturalGas(kBtu)                   0
TotalGHGEmissions                  0
BuildingAge                        0
LastYearENERGYSTARCertified        0
NbYearENERGYSTARCertified          0
dtype: int64

# **II. Définitions des fonctions et des variables globales**


In [7]:
# TARGET =
ID = 'OSEBuildingID'

In [8]:
# préparation des données pour ML, variable a de l'impact ou pas?
 # normalisation des données
 # encodage
 # dataset de training

 # TARGET 1 - prédire une valeur numérique
#tester simple la régression lineaire

#NEW NOTEBOOK
# TARGET 1 - prédire une valeur numérique
#tester simple la régression lineaire

# entrainement modèle, pipeline, grid search
# choix de la méthode de scoring
# comparer les 4 modèles : ElasticNet, SVM, GradientBoosting, RandomForest
# si iverfitting : tester une méthode ensembliste (bagging, boosting)

# check impact de STAR SCORING avec et sans
#c'est pour comparer ce qui est comparable. ENtre BTP aux services similaires (école <> industrie)

#NEW NOTEBOOK
# TARGET 2 - prédire une valeur numérique
# entrainement modèle

#shap
#ElasticNet (lineaire, ), SVM, GradientBoosting, RandomForest).

#pour savoir si une va est importante ?
# simple : utiliser la lib black box
# compliqué : LASSO (connaissance de l'algo!), cas particulier de l'eslatic net, 0 sur tout

Les principaux algorithmes du machine learning supervisé sont les suivants : forêts aléatoires (plusieurs arbres de décisions), arbres de décision, algorithme K-NN (k-Nearest Neighbors) (qui se ressemble.. est identique), régression linéaire, algorithme de Naïve Bayes??, machine à vecteurs de support (SVM)??, régression logistique (hum ?) et boosting de gradient (minimisation de la fonction de coût)

# **III. TARGET 1**

In [9]:
TARGET = 'TotalGHGEmissions'
NUM_FEATURES = ['OSEBuildingID', 'NumberofBuildings', 'NumberofFloors',
       'PropertyGFATotal', 'PropertyGFAParking', 'PropertyGFABuilding(s)',
       'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseTypeGFA',
       'ThirdLargestPropertyUseTypeGFA', 'ENERGYSTARScore', 'SiteEnergyUse(kBtu)',
       'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)',
       'BuildingAge', 'LastYearENERGYSTARCertified',
       'NbYearENERGYSTARCertified']
CAT_FEATURES = [f for f in df.columns if f not in NUM_FEATURES and f != TARGET]

df1 = df.copy()

**Trouver les variables prédictifs à l'aide du lasso ? To do si le temps**

##**A. Entrainement d'un modèle linéaire: LinearRegression**

In [10]:
# importation des librairies
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, PowerTransformer

In [11]:
# séparation des variables explicatives de la variable à prédire
df1 = df.copy()
y = df1.pop(TARGET)
X = df1

In [12]:
# features engineering
y = np.log(y + 1)

preprocessor = ColumnTransformer(
    transformers=[
        ('encoder_nominal', OneHotEncoder(handle_unknown='ignore',sparse_output=False), CAT_FEATURES),
         ('norma', PowerTransformer(), NUM_FEATURES)
    ],
    remainder='drop'
  )

In [13]:
# création de la pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
  ])

In [14]:
# entrainement des modèles selon plusieurs paramètres
# séparation du dataset (entrainement et test)

param_grid = {
    'regressor__fit_intercept' : [False, True],
    'regressor__positive'      : [False, True]
}

gs = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, verbose=1, n_jobs=-1, refit=True, scoring='r2')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
gs.fit(X_train, y_train)
best_model = gs.best_estimator_

print(f'Meilleurs hyperparamètres: {gs.best_params_}')
print(f'Moyenne des scores: {gs.best_score_.round(3)}')
print('----------------')

# model scoring
print(f'Score sur le train : {best_model.score(X_train, y_train).round(3)}')
print(f'Score sur le test : {best_model.score(X_test, y_test).round(3)}')

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Meilleurs hyperparamètres: {'regressor__fit_intercept': False, 'regressor__positive': True}
Moyenne des scores: 0.867
----------------
Score sur le train : 0.957
Score sur le test : 0.9


##**B. Entrainement d'un modèle linéaire: RandomForestRegressor**

In [15]:
from sklearn.ensemble import RandomForestRegressor

In [16]:
# création de la pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
  ])

In [17]:
# entrainement des modèles selon plusieurs paramètres
# séparation du dataset (entrainement et test)

param_grid = {
    'regressor__n_estimators'  : [100, 500],
    # 'regressor__max_depth'     : [2, 3, 4, 5],
    # 'regressor__criterion'     : ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
    # 'regressor__min_samples_split' : [2, 10, 100]
}

gs = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, verbose=1, n_jobs=-1, refit=True, scoring='r2')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
gs.fit(X_train, y_train)
best_model = gs.best_estimator_

print(f'Meilleurs hyperparamètres: {gs.best_params_}')
print(f'Moyenne des scores: {gs.best_score_.round(3)}')
print('----------------')

# model scoring
print(f'Score sur le train : {best_model.score(X_train, y_train).round(3)}')
print(f'Score sur le test : {best_model.score(X_test, y_test).round(3)}')

Fitting 5 folds for each of 2 candidates, totalling 10 fits
Meilleurs hyperparamètres: {'regressor__n_estimators': 500}
Moyenne des scores: 0.979
----------------
Score sur le train : 0.998
Score sur le test : 0.996


##**C. Entrainement d'un modèle linéaire: GradientBoostingRegressor**

In [18]:
from sklearn.ensemble import GradientBoostingRegressor

In [19]:
# création de la pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor())
  ])

In [20]:
# entrainement des modèles selon plusieurs paramètres
# séparation du dataset (entrainement et test)

param_grid = {
    'regressor__learning_rate'      : [0.1, 0.05, 0.5, 1],
    # 'regressor__n_estimators'       : [100, 50, 200, 300],
    # 'regressor__max_depth'          : [3, 2, 4, 5],
    # 'regressor__min_samples_split'  : [2, 3, 4],
    # 'regressor__min_samples_leaf'   : [1, 2, 3],
    # 'regressor__loss'               : ["squared_error", "absolute_error", "huber", "quantile"]
}

gs = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, verbose=1, n_jobs=-1, refit=True, scoring='r2')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
gs.fit(X_train, y_train)
best_model = gs.best_estimator_

print(f'Meilleurs hyperparamètres: {gs.best_params_}')
print(f'Moyenne des scores: {gs.best_score_.round(3)}')
print('----------------')

# model scoring
print(f'Score sur le train : {best_model.score(X_train, y_train).round(3)}')
print(f'Score sur le test : {best_model.score(X_test, y_test).round(3)}')

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Meilleurs hyperparamètres: {'regressor__learning_rate': 0.1}
Moyenne des scores: 0.976
----------------
Score sur le train : 0.998
Score sur le test : 0.995


##**D. Entrainement d'un modèle linéaire: GradientBoostingRegressor**

In [21]:
from sklearn.svm import SVR

In [22]:
# création de la pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR())
  ])

In [23]:
# entrainement des modèles selon plusieurs paramètres
# séparation du dataset (entrainement et test)

param_grid = {
    'regressor__kernel'          : ['rbf'], #, 'linear', 'poly', 'sigmoid', 'precomputed'], #passer dans les dimensions supérieures
    'regressor__degree'         : [3, 2, 4, 5] #for poly
}

gs = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, verbose=1, n_jobs=-1, refit=True, scoring='r2')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
gs.fit(X_train, y_train)
best_model = gs.best_estimator_

print(f'Meilleurs hyperparamètres: {gs.best_params_}')
print(f'Moyenne des scores: {gs.best_score_.round(3)}')
print('----------------')

# model scoring
print(f'Score sur le train : {best_model.score(X_train, y_train).round(3)}')
print(f'Score sur le test : {best_model.score(X_test, y_test).round(3)}')

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Meilleurs hyperparamètres: {'regressor__degree': 3, 'regressor__kernel': 'rbf'}
Moyenne des scores: 0.909
----------------
Score sur le train : 0.962
Score sur le test : 0.951


##**Entrainement du modèle LASSO**


In [ ]:
# On choisie toutes les variables numériques et 3 variables catégorielles
nb_null_value = df.isna().sum(axis=0)
col_to_keep = nb_null_value[nb_null_value < 100].index.to_list()
num_column = df.dtypes[(df.dtypes == 'float64') | (df.dtypes == 'int64')].index.to_list()
cat_to_keep = ['PrimaryPropertyType', 'Neighborhood']
num_to_keep = list(set(col_to_keep).intersection(num_column))
FINAL_COL =  num_to_keep + cat_to_keep #set pour le non itérable, #list pour concat ensuite
FINAL_COL = ['TotalGHGEmissions', 'Electricity(kBtu)', 'SiteEnergyUse(kBtu)', 'NaturalGas(kBtu)'
             , 'NumberofBuildings', 'PrimaryPropertyType', 'Neighborhood']
print((FINAL_COL))

In [ ]:
TARGET = 'TotalGHGEmissions'
FEATURE = [col for col in FINAL_COL if col != TARGET]
df_train = df[FINAL_COL].dropna()
print(FEATURE)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.preprocessing import OneHotEncoder, PowerTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline

In [ ]:
# Features engineering
preprocessor = ColumnTransformer(
    transformers=[
        ('encoder_nominal', OneHotEncoder(handle_unknown='ignore',sparse_output=False), cat_to_keep),
         ('norma', PowerTransformer(), ['Electricity(kBtu)', 'SiteEnergyUse(kBtu)', 'NaturalGas(kBtu)', 'NumberofBuildings'])
    ],
    remainder='drop'
  )

In [ ]:
pp = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', Lasso()),
])

In [ ]:
# On va séparer les variables explicatives de la variable cible.
y = df_train.pop(TARGET)
X = df_train
print(y.shape, X.shape)
# # Question : encodage sur tout le jeu de données ou une partie seulement ? a priori separer puis encoder car pbm de type
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

# X_train_df = pd.DataFrame(data=X_train, columns=FEATURE)
# y_train_df = pd.DataFrame(data=y_train, columns=[TARGET])

# model = PowerTransformer()
# y_train_normed = model.fit_transform(y_train_df)
# X_df = pd.DataFrame(data=X, columns=FEATURE)
# y_df = pd.DataFrame(data=y, columns=[TARGET])
pp.fit(X, y)

In [ ]:
# fit model by using grid search
param_grid = {
    'regressor__alpha'          : [0.0001, 0.001, 0.01, 0.10, 1],
    'regressor__fit_intercept'  : [True, False]
}
grid_search = GridSearchCV(pp, param_grid, cv=5, verbose=1, n_jobs=-1, refit=True, scoring='r2')
grid_search.fit(X, y)
print(grid_search.best_params_)
print('----------------')

# model scoring
best_model = grid_search.best_estimator_
print(f'Model\'s score : {best_model.score(X, y).round(3)}') #est R2 ? Bon va falloir revoir pipeline etc.

In [ ]:
lasso_regressor = best_model.named_steps['regressor']

# Access the coefficients of the Lasso
lasso_coefficients = lasso_regressor.coef_

print("Lasso Coefficients:")
print(lasso_coefficients)

# Print number of total variables and total variables with non null parameter
(
    lasso_coefficients.shape,
    lasso_coefficients[lasso_coefficients != 0.].shape
)
# Le modele a simplifié notre equation: il a choisi seulement 14 variables parmis les 17 !